In [1]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession

In [2]:
win = winlogbeat()
spark = SparkSession.builder.appName("Dataset").config("spark.sql.caseSensitive", "True").getOrCreate()
print(spark)

In [3]:
dataset = win.extract_nested_fields("process_reimaging_2019-09-12174205.json",spark)

[+] Processing a Spark DataFrame..
[+] Reading Mordor file..
[+] Processing Data from Winlogbeat version 7..
[+] DataFrame Returned !


In [4]:
dataset.createOrReplaceTempView("processreimaging")

In [5]:
processreimaging = spark.sql(
'''
SELECT 
    a.computer_name,
    a.OriginalFileName,
    a.LogonId,
    b.ProcessId,
    c.TargetFilename
FROM processreimaging a
JOIN processreimaging b
    ON a.ProcessGuid = b.ProcessGuid
    AND b.channel = "Microsoft-Windows-Sysmon/Operational"
    AND b.event_id = 7
    AND a.OriginalFileName = b.OriginalFileName
    AND a.Image = b.ImageLoaded
JOIN processreimaging c
    ON a.ParentProcessGuid = c.ProcessGuid
    AND c.channel = "Microsoft-Windows-Sysmon/Operational"
    AND c.event_id = 11
    AND a.Image = c.TargetFilename 
WHERE
    a.channel = "Microsoft-Windows-Sysmon/Operational"
    AND a.event_id = 1
    AND a.IntegrityLevel = "High"
    AND LOWER(a.OriginalFileName) != LOWER(substring_index(a.Image, '\\\\', -1))
'''
).show(100,False)

+---------------+----------------+-------+---------+------------------------------------------------------------------------------+
|computer_name  |OriginalFileName|LogonId|ProcessId|TargetFilename                                                                |
+---------------+----------------+-------+---------+------------------------------------------------------------------------------+
|it001.shire.com|svchost.exe     |0xa5b33|8632     |C:\Users\pgustavo\Desktop\CSProcessReimagingPOC\bin\Debug\executing\phase1.exe|
|it001.shire.com|svchost.exe     |0xa5b33|8632     |C:\Users\pgustavo\Desktop\CSProcessReimagingPOC\bin\Debug\executing\phase1.exe|
+---------------+----------------+-------+---------+------------------------------------------------------------------------------+

